In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-20-015-mixed-00409/translator1/fingerprint.pb
/kaggle/input/final-20-015-mixed-00409/translator1/saved_model.pb
/kaggle/input/final-20-015-mixed-00409/translator1/variables/variables.index
/kaggle/input/final-20-015-mixed-00409/translator1/variables/variables.data-00000-of-00001


In [2]:
from nltk.translate.bleu_score import sentence_bleu

In [3]:
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.79.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [5]:
import tensorflow as tf
import tensorflow_text as tf_text

In [20]:
!pip install Metaphone
from metaphone import doublemetaphone

  Preparing metadata (setup.py) ... done
  Created wheel for Metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13919 sha256=99ac418268a76616dc3cfaf272c13f58312933b8d29c69809d3c2adaa7f4ca61
  Stored in directory: /root/.cache/pip/wheels/3a/2c/57/cb7d38aa6c80a870a0341ccf4265c2921fb55b1f7f4b76b46b
Successfully built Metaphone


In [19]:
import pickle
with open('/kaggle/input/hin-vocab/vocab_data.pkl', 'rb') as fp:
    hin_vocab = pickle.load(fp)
vocab_keys=[l for l in hin_vocab]
len(hin_vocab),len(vocab_keys)

(26214, 26214)

In [6]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-015-mixed-00409/translator1')

In [18]:
import re
re.sub("[.!?\\-\'\"]", "", "    kaise ho tum viase .?,KSDFaha   ").lower().strip()

'kaise ho tum viase ,ksdfaha'

In [21]:
def t_text(line):
    line=re.sub("[.!?\\-\'\"]", "",line).lower().strip()
    string=''
    for j in line.split(' '):
        if doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:] in vocab_keys:
            string=string+list(hin_vocab[doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:]])[0]+' '
        else:
            string=string+j+' '
    return string.lower().strip()

In [28]:
three_input_text = tf.constant([t_text('sabse best friend'), 
                                t_text('That toh I know'),
                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhara papa ka naam kya hai'),
])

result = reloaded.tf_translate(three_input_text)

for tr in result['text']:
  print(tr.numpy().decode())

best actor . . . . .
te . mr . toh le my way
she will keep on the shoe of the road . , ,
you tell me
beach . , , childhood you
will father change papa . . . . ? always of us
dear papa ji . . . . . i will become a good king of the night
father . , , i will sleep afternoon after a lot
father will provide a joke with the tank . , ,
forecast for tshirt like punjab . . .
, dear papa ji . . . left your drive


In [ ]:
sentence_bleu(reference, candidate )